# Exercise

**file name:** clustering_anomaly_detection.py or clustering_anomaly_detection.ipynb




### Clustering - DBSCAN

Ideas: 

Use DBSCAN to detect anomalies in curriculumn access. 


In [1]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import seaborn as sns
import env

### Acquire and prepare data

In [2]:
url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/curriculum_logs'

query = '''
SELECT logs.date,
	   logs.time,
       logs.path as endpoint,
       logs.user_id,
       logs.cohort_id,
       logs.ip as source_ip,
       cohorts.name as cohort_name,
       cohorts.start_date as cohort_start,
       cohorts.end_date as cohort_end,
       cohorts.program_id as program_id
FROM logs
JOIN cohorts ON logs.cohort_id= cohorts.id;
'''

df = pd.read_sql(query, url)
df.head()

,date,time,endpoint,user_id,cohort_id,source_ip,cohort_name,cohort_start,cohort_end,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,2018-01-08,2018-05-17,2


In [3]:
num_programs = pd.DataFrame(df.groupby(by='user_id').cohort_id.nunique())
df2 = df.merge(num_programs, how='inner', on='user_id')
df2.rename(columns={'cohort_id_y': 'number_of_classes'}, inplace=True)
df=df2.copy()

In [4]:
df.head()

,date,time,endpoint,user_id,cohort_id_x,source_ip,cohort_name,cohort_start,cohort_end,program_id,number_of_classes
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,2
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,2
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,2
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,2
4,2018-01-26,10:40:15,javascript-i/functions,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,2


In [5]:
# Concatenate date and time to new column so data can be explored more granularly
df['date_time']=df.date+' '+df.time

df.date = pd.to_datetime(df.date)
df.cohort_start = pd.to_datetime(df.cohort_start)
df.cohort_end = pd.to_datetime(df.cohort_end)
df.date_time = pd.to_datetime(df.date_time)
df = df.set_index(df.date_time)

In [6]:
#Add column for days after graduation
df['days_after_grad']=df.date-df.cohort_end

valmap = {2:'java_web_dev', 1:'php_web_dev', 3:'data_science', 4:'front_end_web_dev'}
df['program']=df['program_id'].map(valmap)

In [7]:
df.head()


,date,time,endpoint,user_id,cohort_id_x,source_ip,cohort_name,cohort_start,cohort_end,program_id,number_of_classes,date_time,days_after_grad,program
date_time,,,,,,,,,,,,,,
2018-01-26 09:55:03,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,2,2018-01-26 09:55:03,720 days,web_dev
2018-01-26 09:56:02,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,2,2018-01-26 09:56:02,720 days,web_dev
2018-01-26 09:56:05,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,2,2018-01-26 09:56:05,720 days,web_dev
2018-01-26 09:56:06,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,2,2018-01-26 09:56:06,720 days,web_dev
2018-01-26 10:40:15,2018-01-26,10:40:15,javascript-i/functions,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,2,2018-01-26 10:40:15,720 days,web_dev


In [8]:
# Add program description
valmap = {2:'java_web_dev', 1:'php_web_dev', 3:'data_science', 4:'front_end_web_dev'}
df['program']=df['program_id'].map(valmap)

In [9]:
df.sample(25)

,date,time,endpoint,user_id,cohort_id_x,source_ip,cohort_name,cohort_start,cohort_end,program_id,number_of_classes,date_time,days_after_grad,program
date_time,,,,,,,,,,,,,,
2018-08-18 19:24:14,2018-08-18,19:24:14,mysql,1,8.0,72.177.226.58,Hampton,2015-09-22,2016-02-06,1,2,2018-08-18 19:24:14,924 days,php_web_dev
2020-02-18 14:20:08,2020-02-18,14:20:08,4-python/3-data-types-and-variables,575,55.0,97.105.19.58,Curie,2020-02-03,2020-07-07,3,1,2020-02-18 14:20:08,-140 days,data_science
2019-04-29 09:12:05,2019-04-29,09:12:05,jquery/selectors,377,31.0,97.105.19.58,Andromeda,2019-03-18,2019-07-30,2,1,2019-04-29 09:12:05,-92 days,java_web_dev
2019-07-08 09:04:26,2019-07-08,09:04:26,toc,384,31.0,97.105.19.58,Andromeda,2019-03-18,2019-07-30,2,1,2019-07-08 09:04:26,-22 days,java_web_dev
2019-12-18 12:00:15,2019-12-18,12:00:15,mysql/database-design,490,51.0,97.105.19.58,Deimos,2019-09-16,2020-02-27,2,1,2019-12-18 12:00:15,-71 days,java_web_dev
2019-10-31 09:37:25,2019-10-31,09:37:25,content/laravel/quickstart/db-migrations.html,68,13.0,68.65.212.72,Kings,2016-05-23,2016-09-15,1,1,2019-10-31 09:37:25,1141 days,php_web_dev
2019-07-18 15:50:47,2019-07-18,15:50:47,9-anomaly-detection/1-overview,11,28.0,97.105.19.58,Staff,2014-02-04,2014-02-04,2,2,2019-07-18 15:50:47,1990 days,java_web_dev
2020-08-06 16:32:52,2020-08-06,16:32:52,sql/mysql-overview,581,55.0,70.112.179.142,Curie,2020-02-03,2020-07-07,3,2,2020-08-06 16:32:52,30 days,data_science
2018-08-02 18:20:40,2018-08-02,18:20:40,java-ii/collections,142,22.0,72.181.104.64,Teddy,2018-01-08,2018-05-17,2,1,2018-08-02 18:20:40,77 days,java_web_dev


Use DBSCAN to detect anomalies in other products from the customers dataset. 



Use DBSCAN to detect anomalies in number of bedrooms and finished square feet of property for the filtered dataset you used in the clustering project (single unit properties with a logerror).
